In [78]:
from Doc2VecUtils import SentimentDocument
from Doc2VecUtils import file_len
import os.path

In [79]:
import gensim.utils
def read_corpus_plain(fname):
    for line in open(fname, encoding='utf-8'):
        # For training data, add tags
        tokens = gensim.utils.to_unicode(line).split()
        if(len(tokens)==0):
            continue
        words = tokens[1:]
        yield ' '.join(words)

In [80]:
dirname = 'data_by_returns_small'
num_lines_test = file_len(os.path.join(dirname, 'test-pos.txt'))
num_lines_test += file_len(os.path.join(dirname, 'test-neg.txt'))
num_lines_train = file_len(os.path.join(dirname, 'train-pos.txt'))
num_lines_train += file_len(os.path.join(dirname, 'train-neg.txt'))

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
vectorizer = TfidfVectorizer(max_features=300, max_df=0.3, min_df=0.1, sublinear_tf=True)
# train_docs = read_corpus_plain(os.path.join(dirname, 'train-all.txt'))
# tfidf_matrix = vectorizer.fit_transform(train_docs)
# feature_names = vectorizer.get_feature_names()
# dense = tfidf_matrix.todense()
# denselist = dense.tolist()
# df = pd.DataFrame(denselist, columns=feature_names)
# df

In [117]:
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from sklearn.utils import resample
from Doc2VecUtils import read_labeled_corpus
# subsample = 1.0

train_set = read_labeled_corpus(os.path.join(dirname, 'train-pos.txt'), os.path.join(dirname, 'train-neg.txt'), 'train')
train_targets, train_words = zip(*[(doc.sentiment, ' '.join(doc.words)) for doc in train_set])
train_regressors = vectorizer.fit_transform(train_words).todense().tolist()
# train_targets, train_regressors = resample(train_targets, train_regressors, n_samples = int(subsample * num_lines_train))
# embedded = TSNE(n_components=2).fit_transform(train_regressors)
# vis_x = embedded[:, 0]
# vis_y = embedded[:, 1]

# plt.scatter(vis_x, vis_y, c=train_targets, cmap=plt.cm.get_cmap("Paired"))
# plt.colorbar(ticks=range(2))
# plt.title("TSNE of Tfidf Vectors")
# plt.show()

In [118]:
from sklearn import linear_model
from sklearn import ensemble
from sklearn import svm
import numpy as np

In [119]:
results = []
#classifiers = [linear_model.LogisticRegression(), ensemble.RandomForestClassifier(), svm.SVC()]
classifiers = [linear_model.LogisticRegression(C=1.5)]

valid_set = read_labeled_corpus(os.path.join(dirname, 'valid-pos.txt'), os.path.join(dirname, 'valid-neg.txt'), 'valid')
valid_targets, valid_words = zip(*[(doc.sentiment, ' '.join(doc.words)) for doc in valid_set])
valid_regressors = vectorizer.fit_transform(valid_words).todense().tolist()
out_string = ""
for classifier in classifiers:
    classifier.fit(train_regressors, train_targets)
    valid_predictions = classifier.predict(valid_regressors)
    len_predictions = len(valid_predictions)
    corrects = sum(np.rint(valid_predictions) == valid_targets)
    errors = len_predictions - corrects
    error_rate = float(errors) / len_predictions
    err, err_count, valid_count, predictor = error_rate, errors, len_predictions, classifier
    results.append((1-err, valid_count-err_count, valid_count, predictor, "TfidfVectorizer(max_features=300, max_df=0.3, min_df=0.1, sublinear_tf=True)"))
    out_string+="Error: {0}; Error Count: {1}; Test Count {2}; Predictor: {3}; Model: {4}\n".format(err, err_count, valid_count, predictor, "TfidfVectorizer(max_features=300, max_df=0.3, min_df=0.1, sublinear_tf=True)")
df = pd.DataFrame(data=results, columns=['Accuracy', 'Number Correct', 'Test Size', 'Classifier', 'Model'])
df
# Best accuracy: .517982 C=1.5 Tfidf max_features=300 max_df=0.3 min_df=0.1 sublinear_tf=True => Test accuracy=.508390
# Best accuracy: .611389 C=1.5 Tfidf max_features=800 max_df=0.3 min_df=0.1 sublinear_tf=True => Test accuracy=.497322

,Accuracy,Number Correct,Test Size,Classifier,Model
0,0.517982,1037,2002,"LogisticRegression(C=1.5, class_weight=None, d...","TfidfVectorizer(max_features=300, max_df=0.3, ..."


In [120]:
print(out_string)

Error: 0.482017982017982; Error Count: 965; Test Count 2002; Predictor: LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False); Model: TfidfVectorizer(max_features=300, max_df=0.3, min_df=0.1, sublinear_tf=True)



In [121]:
results = []

test_set = read_labeled_corpus(os.path.join(dirname, 'test-pos.txt'), os.path.join(dirname, 'test-neg.txt'), 'test')
test_targets, test_words = zip(*[(doc.sentiment, ' '.join(doc.words)) for doc in test_set])
test_regressors = vectorizer.fit_transform(test_words).todense().tolist()
out_string = ""
test_predictions = classifier.predict(test_regressors)
len_predictions = len(test_predictions)
corrects = sum(np.rint(test_predictions) == test_targets)
errors = len_predictions - corrects
error_rate = float(errors) / len_predictions
err, err_count, test_count, predictor = error_rate, errors, len_predictions, classifier
results.append((1-err, test_count-err_count, test_count, predictor, "TfidfVectorizer(max_features=300, max_df=0.3, min_df=0.1, sublinear_tf=True)"))
out_string+="Error: {0}; Error Count: {1}; Test Count {2}; Predictor: {3}; Model: {4}\n".format(err, err_count, test_count, predictor, "TfidfVectorizer(max_features=300, max_df=0.3, min_df=0.1, sublinear_tf=True)")
df = pd.DataFrame(data=results, columns=['Accuracy', 'Number Correct', 'Test Size', 'Classifier', 'Model'])
df

,Accuracy,Number Correct,Test Size,Classifier,Model
0,0.50839,1424,2801,"LogisticRegression(C=1.5, class_weight=None, d...","TfidfVectorizer(max_features=300, max_df=0.3, ..."


In [122]:
print(out_string)

Error: 0.4916101392359871; Error Count: 1377; Test Count 2801; Predictor: LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False); Model: TfidfVectorizer(max_features=300, max_df=0.3, min_df=0.1, sublinear_tf=True)

